# Allgemein

In [1]:
import random
random.seed(1)

import IPython.display 
import ipycanvas as cnv

In [2]:
def play_game(canvas):
    State = gStart
    while True: 
        val, State = best_move(State);
        draw(State, canvas, f'For me, the game has the value {val}.')
        if finished(State):
            final_msg(State)
            return
        IPython.display.clear_output(wait=True)
        State = get_move(State)
        draw(State, canvas, '')
        if finished(State):
            IPython.display.clear_output(wait=True)
            final_msg(State)
            return

In [3]:
def to_list(State): 
    return [list(row) for row in State]

def to_tuple(State): 
    return tuple(tuple(row) for row in State)

# Memoisierung

In [4]:
gCache = {}

def memoize(f):
    global gCache
    
    def f_memoized(*args):
        if (f, args) in gCache:
            return gCache[(f, args)]
        result = f(*args)
        gCache[(f, args)] = result
        return result
    
    return f_memoized

# Mini-Max-Algorithmus

In [5]:
def best_move(State):
    NS        = next_states(State, gPlayers[0])
    bestVal   = maxValue(State)
    BestMoves = [s for s in NS if minValue(s) == bestVal]
    BestState = random.choice(BestMoves)
    return bestVal, BestState

In [6]:
def maxValue(State):
    if finished(State):
        return utility(State)
    return max([ minValue(ns) for ns in next_states(State, gPlayers[0]) ])

In [7]:
def minValue(State):
    if finished(State):
        return utility(State)
    return min([ maxValue(ns) for ns in next_states(State, gPlayers[1]) ])

## Beispiel Tic-Tac-Toe

In [8]:
gPlayers = [ 'X', 'O' ]
gStart = tuple( tuple(' ' for col in range(3)) for row in range(3) )
gAllLines = [ [ (row, col) for col in range(3) ] for row in range(3) ] \
          + [ [ (row, col) for row in range(3) ] for col in range(3) ] \
          + [ [ (0, 0), (1, 1), (2, 2) ] ]                             \
          + [ [ (0, 2), (1, 1), (2, 0) ] ]

def next_states(State, player):
    Result = []
    for row in range(3):
        for col in range(3):
            if State[row][col] == ' ':
                NextState           = to_list(State)
                NextState[row][col] = player
                NextState           = to_tuple(NextState)
                Result.append(NextState)
    return Result

def utility(State):
    for Line in gAllLines:
        Marks = { State[row][col] for row, col in Line }
        if len(Marks) == 1 and  Marks != { ' ' }: 
            if Marks == { 'X' }:
                return  1
            else:
                return -1
    for row in range(3):
        for col in range(3):
            if State[row][col] == ' ':
                return None  # the board is not filled  
    # at this point, the board has been filled, but there is no winner, hence it's a draw
    return 0

def finished(State): 
    return utility(State) != None

### Spiel

In [9]:
g_size = 150

def create_canvas():
    n = 3
    canvas = cnv.Canvas(size=(g_size * n, g_size * n + 50))
    display(canvas)
    return canvas

def get_move(State):
    State = to_list(State)
    while True:
        try:
            row, col = input('Enter move here: ').split(',')
            row, col = int(row), int(col)
            if State[row][col] == ' ':
                State[row][col] = 'O'
                return to_tuple(State)
            print("Don't cheat! Please try again.")  
        except:
            print('Illegal input.')  
            print('row and col are numbers from the set {0,1,2}.')

def final_msg(State):
    if finished(State):
        if utility(State) == -1:
            print('You have won!')
        elif utility(State) == 1:
            print('The computer has won!')
        else:
            print("It's a draw.")
        return True
    return False

def draw(State, canvas, value):
    canvas.clear()
    n = len(State)
    canvas.font = '90px sans-serif'
    canvas.text_align    = 'center'
    canvas.text_baseline = 'middle'
    for row in range(n):
        for col in range(n):
            x = col * g_size
            y = row * g_size
            canvas.line_width = 3.0
            canvas.stroke_rect(x, y, g_size, g_size)
            symbol = State[row][col]
            if symbol != ' ':
                x += g_size // 2
                y += g_size // 2
                if symbol == 'X':
                    canvas.fill_style ='red'
                else:
                    canvas.fill_style ='blue'
                canvas.fill_text(symbol, x, y)
    canvas.font = '12px sans-serif'
    canvas.fill_style = 'green'
    for row in range(n):
        for col in range(n):
            x = col * g_size + 16
            y = row * g_size + 141
            canvas.fill_text(f'({row}, {col})', x, y)            
    canvas.font = '20px sans-serif'
    canvas.fill_style = 'black'
    x = 1.5 * g_size
    y = 3.2 * g_size
    canvas.fill_text(str(value), x, y)

def play_game(canvas):
    State = gStart
    while True: 
        val, State = best_move(State);
        draw(State, canvas, f'For me, the game has the value {val}.')
        if finished(State):
            final_msg(State)
            return
        IPython.display.clear_output(wait=True)
        State = get_move(State)
        draw(State, canvas, '')
        if finished(State):
            IPython.display.clear_output(wait=True)
            final_msg(State)
            return

In [10]:
# canvas = create_canvas()
# val = maxValue(gStart)
# draw(gStart, canvas, f'Current value of game for "X": {val}')
# play_game(canvas)

### Ohne vs. mit Memoizierung

In [11]:
%%time
val = maxValue(gStart)
val

CPU times: total: 6.22 s
Wall time: 6.24 s


0

In [12]:
oldMaxValue = maxValue
oldMinValue = minValue

maxValue = memoize(maxValue)
minValue = memoize(minValue)

In [13]:
%%time
val = maxValue(gStart)
val

CPU times: total: 109 ms
Wall time: 118 ms


0

In [14]:
maxValue = oldMaxValue
minValue = oldMinValue

### Ohne vs. mit Bitboard

In [15]:
%%time
val = maxValue(gStart)
val

CPU times: total: 6.8 s
Wall time: 6.97 s


0

In [16]:
gPlayers = [0, 1]
gStart = 0

def set_bits(Bits):
    result = 0
    for b in Bits:
        result |= 1 << b # bitwise or 2**b
    return result

def set_bit(n): 
    return 1 << n

def empty(state):
    Free  = { n for n in range(9) }
    Free -= { n for n in range(9) if state & (1 << n) != 0 } # Spieler X => von Bit 0 - 8
    Free -= { n for n in range(9) if state & (1 << (9 + n)) != 0 } # Spieler O => von Bit 9 - Bit 17
    return Free

def next_states(state, player):
    Empty  = empty(state)
    Result = []
    for n in Empty:
        next_state = state | set_bit(player * 9 + n)
        Result.append(next_state)
    return Result

gAllLines = [ set_bits([0,1,2]), # 1st row
              set_bits([3,4,5]), # 2nd row
              set_bits([6,7,8]), # 3rd row
              set_bits([0,3,6]), # 1st column
              set_bits([1,4,7]), # 2nd column
              set_bits([2,5,8]), # 3rd column
              set_bits([0,4,8]), # falling diagonal
              set_bits([2,4,6]), # rising diagonal
            ]

def utility(state):
    for mask in gAllLines:
        if state & mask == mask:
            return 1               # the computer has won
        if (state >> 9) & mask == mask:
            return -1              # the computer has lost
    # 511 == 2**9 - 1 = 0b1_1111_1111
    # state & 511: Ergibt gesetzten Bits von Spieler X
    # (state & 511) | (state >> 9): Schiebt state um 9 nach Rechts (Bits von Spieler 0) und verknüpft Wert mit vorherigem Ergebnis mit Bitwise OR
    # Falls das Ergebnis 511 Ergeben würde, wären alle Felder von einem Spieler belegt
    if (state & 511) | (state >> 9) != 511: # the board is not yet filled
        return None
    # at this point, the board has been filled, but there is no winner hence its a draw
    return 0 # it's a draw

In [17]:
%%time
val = maxValue(gStart)
val

CPU times: total: 3.19 s
Wall time: 3.21 s


0